In [3]:
# !pip install transformers datasets

In [77]:
import torch
import torch.nn.functional as F
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from datasets import load_dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)  # For binary classification
model.to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predicted class: 0


### Tokenize the dataset

In [ ]:
dataset = load_dataset("glue", "sst2")

train_data = dataset["train"]
validation_data = dataset["validation"]
test_data = dataset["test"]

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [ ]:
train_data[0]

{'sentence': 'hide new secretions from the parental units ',
 'label': 0,
 'idx': 0}

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = [[] for _ in range(6)]
for split, xb, yb in zip([train_data, validation_data, test_data], [X_train, X_val, X_test], [y_train, y_val, y_test]):
    for observation in split:
        sentence, label, _ = observation.values()
        xb.append(sentence)
        yb.append(label)
    
    xb = tokenizer(xb, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)

In [39]:
X_train = tokenizer(X_train, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
X_val = tokenizer(X_val, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
X_test = tokenizer(X_test, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)

In [46]:
X_train.keys()

dict_keys(['input_ids', 'attention_mask'])

In [48]:
X_train['input_ids'].shape

torch.Size([67349, 67])

In [ ]:
batch_size = 32
eval_iters = 200

In [88]:
def get_batch(split):
    data = X_train if split == 'train' else X_val
    label = y_train if split == 'train' else y_val
    idx = torch.randint(low=0, high=data['input_ids'].shape[0], size=(batch_size,))
    xb = {k: v[idx] for k, v in data.items()}
    yb = torch.tensor(label)[idx].to(device)
    return xb, yb


In [89]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        accuracies = torch.clone(losses)

        for k in range(eval_iters):
            xb, yb = get_batch(split)
            logits = model(**xb)['logits']
            losses[k] = F.cross_entropy(logits, yb)

            prob = F.softmax(logits, dim=1)
            label = torch.argmax(prob, dim=1)
            accuracy = torch.sum(label==yb) / len(yb)
            accuracies[k] = accuracy
        out[split] = losses.mean()
        out[f'{split} accuracy'] = accuracies.mean()
    model.train()
    return out

In [90]:
estimate_loss()

{'train': tensor(0.7785),
 'train accuracy': tensor(0.4484),
 'val': tensor(0.7504),
 'val accuracy': tensor(0.4906)}

In [60]:
xb, yb = get_batch('train')

In [ ]:
kktorch.zeros(eval_iters).copy()

AttributeError: 'Tensor' object has no attribute 'copy'